In [17]:
import pandas as pd
import tqdm
import glob
import os

In [18]:
files = glob.glob("../data/wikipedia/a.parquet")
files = [f for f in files if "all" not in f]

In [33]:
chunk_size = 10000000

In [34]:
def output_seperated_files(token_length, stride, ):    
    output_dir = f"../data/wikipedia/sep_chunk/token_length{token_length}_stride_sentence{stride}"
    os.makedirs(output_dir, exist_ok=True)
    chunk = 1
    df_wiki_sep = []
    for file in files:
        df_wiki = pd.read_parquet(file)
        df_wiki = df_wiki.drop("categories", axis=1)
        for i in tqdm.tqdm(range(len(df_wiki))):
            series = df_wiki.iloc[i]
            text = series["text"]

            text_space = text.split(".")
            text = ""
            text_stride = []
            
            for i in range(0, len(text_space)):
                series_ = series.copy()
                text += text_space[i] + "."
                text_stride.append(text_space[i])
                text_stride = text_stride[-stride:]
                
                if len(text.split(" ")) > token_length:
                    series_["text"] = text
                    df_wiki_sep.append(series_)
                    text = ".".join(text_stride)
            if len(df_wiki_sep) > chunk_size:
                print(f"chunk {chunk}")
                df_wiki_sep = pd.DataFrame(df_wiki_sep)
                df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
                df_wiki_sep = []
                chunk += 1
    df_wiki_sep = pd.DataFrame(df_wiki_sep)
    df_wiki_sep.to_parquet(f"{output_dir}/chunk{chunk}.parquet")
    

In [35]:
output_seperated_files(token_length=90, stride=3)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 442726/442726 [07:02<00:00, 1047.33it/s]
